## IR基底とサンプリングポイントの生成

In [ ]:
using PyCall
#run(`$(PyCall.python) -m pip install --upgrade cython`)
#run(`$(PyCall.python) -m pip install plfit`)
#plfit= pyimport("plfit")
#Pkg.add("Conda")
PyCall.pyprogramname
using Conda
run(`$(PyCall.python) -m pip install irbasis3`)
#Conda.pip_interop(true, ROOTENV)
#Conda.pip("install", "irbasis3==3.0a4")
#Conda.add("plfit")
#PyCall.pyprogramname
#pip install irbasis3==3.0a4
#import PyCall: pyimport
#irbasis3 = pyimport("irbasis3")
import PyCall: pyimport
irbasis3 = pyimport("irbasis3")
const newaxis = [CartesianIndex()]

In [ ]:

lambda_ = 1e+5
beta = 1e+3
eps = 1e-7
# Number of divisions along each reciprocal lattice vector
# Note: For a smaller nk (e.g., 64), an unphysical small structures appear in the self-energy at low frequencies.
K_size = 256
U = 2.0 # Onsite repulsion

wmax = lambda_/beta
K = irbasis3.KernelFFlat(lambda_=lambda_)
basis = irbasis3.FiniteTempBasis(K, statistics="F", beta=beta, eps=eps)
L = basis.size

# Sparse sampling in tau
smpl_tau = irbasis3.TauSampling(basis)
ntau = length(smpl_tau.sampling_points)
println("cond (tau): ", smpl_tau.cond)

# Sparse sampling in Matsubara frequencies
smpl_matsu = irbasis3.MatsubaraSampling(basis)
nw = length(smpl_matsu.sampling_points)
println("cond (matsu): ", smpl_matsu.cond)

## 2. 波数空間上でのnon-interactingのGreen関数の生成

In [ ]:
#tupleの生成
kps = (K_size, K_size)

#K_sizeの二乗
nk = prod(kps)
nr = nk

ek = zeros(Float64, kps...) # Equivalent to zeros(Float64, kps[1], kps[2])
for k2 in 1:K_size, k1 in 1:K_size
   ek[k1, k2] = -2 * (cos(2*π*k1/K_size) + cos(2*π*k2/K_size))
end

In [ ]:
iv = (im * π/beta) * smpl_matsu.sampling_points
gkf = 1.0 ./ (iv[:,newaxis] .- vec(ek)[newaxis,:])

using PyPlot

rcParams = PyPlot.PyDict(PyPlot.matplotlib."rcParams")

#rcParams["text.usetex"] = true
rcParams["font.family"] = "serif"
rcParams["font.size"] = 16
#plot()
plt.plot(smpl_matsu.sampling_points, imag(gkf[:,1]), label="k=Γ", marker="x")
#plt.xlabel(L"$\nu$")
#plt.ylabel(L"$\mathrm{Im}~G(\mathrm{i}\nu)$")
#plt.legend()

In [ ]:
gkl = smpl_matsu.fit(gkf, axis=0)

@assert size(gkl) == (L, nk)

plt.semilogy(abs.(gkl[:,1]))
plt.semilogy(basis.s, ls="--")
plt.xlabel(L"$l$")
plt.ylabel(L"$|G(l, \Gamma)|$")